<a href="https://colab.research.google.com/github/lamld203844/Biomedical-Image-Processing/blob/main/baseline_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recap: EDA
- 78 Video:
  - 6254 frame images
  - 6247 mask images
  - 6224 mask enhance images
- Frame label count: (6, 3743), (5, 1136), (3, 1022), (4, 323)
  - Label 3-None, 4-OnlySP, 5-OnlyHead, 6-SP+Head in framelabel.csv

In [1]:
# Google colab
from google.colab import drive
drive.mount('/content/drive')

# Kaggle

# Local

Mounted at /content/drive


In [4]:
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00


In [5]:
%load_ext watermark
%watermark -v -p numpy,pandas,matplotlib,seaborn,torch

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

numpy     : 1.23.5
pandas    : 1.5.3
matplotlib: 3.7.1
seaborn   : 0.12.2
torch     : 2.1.0+cu121



In [6]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Prepare dataset

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_names = [filename for filename in os.listdir(image_dir) if filename.endswith(".png")]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = os.path.join(self.image_dir, image_name)
        mask_path = os.path.join(self.mask_dir, image_name.replace(".png", "_mask.png"))

        image = plt.imread(image_path)
        mask = plt.imread(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        # Convert mask to appropriate format (e.g., LongTensor for binary segmentation)
        mask = torch.as_tensor(mask, dtype=torch.long)

        return image, mask

In [ ]:



# Define transformations if needed
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Adjust these dimensions as needed
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Create training and validation datasets
train_dataset = SegmentationDataset(train_image_dir, train_mask_dir, transform=transform)
val_dataset = SegmentationDataset(val_image_dir, val_mask_dir, transform=transform)

# Create PyTorch Dataloaders with batch size and shuffling
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4)

# Example usage for accessing a batch of data
images, masks = next(iter(train_dataloader))

